In [ ]:
import math
import time
import numpy as np
import json
import glob
import warnings
import requests

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None)


import datetime

df = pd.read_pickle('dfipynb.pickle')
dftmp=df

def anpassung_zeit(df):
    df = df[df['Querschnitt'].notna()]
    df = df[df['Datum'].notna()]
    df.HHMM = df.HHMM / 100
    df['HHMM']=df['HHMM'].replace(24.0,23.59)
    df.head(100)
    df['Datum und Zeit'] = df["Datum"].astype(str).str[:10]+ ' ' +df.HHMM.astype(str)
    df['Datumszeit'] = pd.to_datetime(df['Datum und Zeit'], format='%Y-%m-%d %H.%M', errors= "coerce")
    return df
df=anpassung_zeit(df)

In [ ]:
import pandas as pd
import pandasql as psql



def addAggregationsForEachDay(df):
    carsPerHour = "SELECT Zst, Datumszeit, sum(Querschnitt) as anzahlZurStunde FROM df GROUP BY Zst, Datumszeit"
    df = psql.sqldf(carsPerHour)

    zstMeasruesPerDay = "SELECT Zst, cast(Datumszeit as date) as Datum, sum(anzahlZurStunde) as amountOnThisDay, sum(anzahlZurStunde)/24 as avgPerHourOnThisDay, min(anzahlZurStunde) as lowestAmountOnThisDay, max(anzahlZurStunde) as highestAmountOnThisDay " \
                        "FROM df " \
                        "GROUP BY Zst, Datum"
    df = psql.sqldf(zstMeasruesPerDay)
    return df

addAggregationsForEachDay(df)



In [ ]:
temp = df.head(1000)
sql1 =  "SELECT Zst, Datumszeit, sum(Querschnitt) as anzahlZurStunde FROM temp GROUP BY Zst, Datumszeit"
temp = psql.sqldf(sql1)

convertToDate= "Select cast(Datumszeit as date) as Datum from temp"
psql.sql(convertToDate)

sql2 = "SELECT Zst, cast(Datumszeit as date) as Datum, sum(anzahlZurStunde) as amountOnThisDay, sum(anzahlZurStunde)/24 as avgPerHourOnThisDay, min(anzahlZurStunde) as lowestAmountOnThisDay, max(anzahlZurStunde) as highestAmountOnThisDay " \
                        "FROM temp " \
                        "GROUP BY Zst, cast(Datumszeit as date)"

temp = psql.sqldf(sql2)

In [ ]:
temp = df.head(1000)
temp = psql.sqldf(sql1)
temp



In [ ]:
import plotly.express as px
fig = px.line(df, y='Querschnitt',x="Datumszeit", hover_data=["Datumszeit"], color="Zst")
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()